In [1]:
from ultralytics import YOLO
import cv2
import math 
import numpy as np
import requests
from PIL import ImageDraw ,Image

In [2]:
import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 50  # Set Duration To 1000 ms == 1 second
# winsound.Beep(frequency, duration)

In [5]:
url = 'http://10.232.141.63:8080//shot.jpg'
while True:
    img = requests.get(url)
    img = cv2.imdecode(np.array(bytearray(img.content),dtype=np.uint8),-1)
    img = cv2.resize(img,(640,480))
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    # Range for lower red
    lower_red1 = np.array([0,70,50])
    upper_red1 = np.array([20,255,150])
    mask0 = cv2.inRange(hsv, lower_red1, upper_red1)
    
    # range for upper red
    lower_red2 = np.array([220,70,50])
    upper_red2 = np.array([255,255,150])
    mask1 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask0+mask1
    output = cv2.bitwise_and(img, img, mask=mask)
    numpy_horizontal = np.hstack((img, output))
    corrosion_pixel = np.count_nonzero(mask)
    if corrosion_pixel > 640*640*0.0025:
        corrosion = 1
        winsound.Beep(frequency, duration)
        
    else :
        corrosion = 0 
    final = cv2.putText(numpy_horizontal, str(corrosion), (50,50), cv2.FONT_HERSHEY_SIMPLEX , 2, (255, 0, 0) , 2, cv2.LINE_AA) 
    
    cv2.imshow('Webcam', final)

    if cv2.waitKey(1) == ord('q'):
        break

# cap.release()
cv2.destroyAllWindows()

In [10]:
# model
model = YOLO('final_weights/detection.pt')
# model.to('cuda')

cam = cv2.VideoCapture(1) 
  
# reading the input using the camera 
 
while True:
    result, img = cam.read()
    img = cv2.resize(img,(640,480))
    results = model(img,conf=0.2,verbose=False)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            # class name
            cls = int(box.cls[0])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, f'corrosion {confidence}', org, font, fontScale, color, thickness)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

# cap.release()
cv2.destroyAllWindows()

In [9]:
# model
model = YOLO('final_weights/segmentation.pt')
# model.to('cuda')

while True:
    img = requests.get(url)
    img = cv2.imdecode(np.array(bytearray(img.content),dtype=np.uint8),-1)
    img = cv2.resize(img,(640,480))
    results = model.predict(np.array(img),conf=0.8,verbose=False)
    
    # coordinates
    for r in results:
        masks = r.masks
        if masks != None :
            for mask in masks:
                # bounding box
                polygon = mask.xy[0]
                
                drawn_img = cv2.polylines(img,np.int64([polygon]),isClosed = True,color = (255, 0, 0),thickness = 2)
                cv2.imshow('Webcam', drawn_img)
        else :
            cv2.imshow('Webcam',img)
    
    if cv2.waitKey(1) == ord('q'):
        break

# cap.release()
cv2.destroyAllWindows()

NameError: name 'url' is not defined